In [ ]:
# Exam 14th of January 2020 Solutions for 1MS041

# Problem 1: Singular Value Decomposition (SVD)
from sympy import Matrix, sqrt

M = Matrix([[1, 1], [0, 3], [3, 0]])
rank = M.rank()

# Compute SVD manually (eigenvalue decomposition for symmetric M.T * M and M * M.T)
eigenvalues_v = M.T * M
eigenvalues_u = M * M.T

# Singular values
singular_values = [sqrt(val) for val in eigenvalues_v.eigenvals().keys()]
D = Matrix.diag(*singular_values)

# Eigenvectors (normalized for U and V)
U = eigenvalues_u.eigenvects()[0][2][0].normalized()
V = eigenvalues_v.eigenvects()[0][2][0].normalized()

print(f"Rank: {rank}")
print(f"U: {U}")
print(f"D: {D}")
print(f"V: {V}")

# Problem 2: Wald Test
import numpy as np

dataSamples2 = np.array([-10, 10, 10, -10, -10, -10, 10, 10, -10, -10, -10, 10, 10, -10, 10, 10, 10])

# Compute MLE for theta
count_10 = sum(dataSamples2 == 10)
count_neg10 = sum(dataSamples2 == -10)
thetaHat = count_10 / len(dataSamples2)

# Null hypothesis value
NullTheta = 0.5

# Standard error (sqrt[Fisher Information])
n = len(dataSamples2)
seTheta = np.sqrt((NullTheta * (1 - NullTheta)) / n)

# Wald Statistic
W = (thetaHat - NullTheta) / seTheta
rejectNull1 = abs(W) > 2.0
print("MLE thetaHat =", thetaHat)
print("Wald statistic =", W)
if rejectNull1:
    print("Reject the null hypothesis that theta_0=0.5")
else:
    print("Fail to reject the null hypothesis that theta_0=0.5")

# Problem 3: Residual Analysis
from scipy.linalg import lstsq
import matplotlib.pyplot as plt

# Mock example data
eqDepth = np.linspace(1, 10, 20)
eqMagnitude = 2.5 + 0.75 * eqDepth + np.random.normal(0, 0.5, size=len(eqDepth))

# Fit least squares model
M1 = np.vstack([np.ones_like(eqDepth), eqDepth]).T
b, *_ = lstsq(M1, eqMagnitude)

# Residual plot
residuals = eqMagnitude - (b[0] + b[1] * eqDepth)
plt.scatter(eqDepth, residuals)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("Depth")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

# Wald Test for beta_1=0
se_beta1 = np.sqrt(np.var(residuals) / (eqDepth - eqDepth.mean())**2).sum()
W_beta1 = b[1] / se_beta1
RejectNullHypothesisForProblem4 = abs(W_beta1) > 1.96
print("Reject Null Hypothesis for beta_1=0:", RejectNullHypothesisForProblem4)

# Problem 4: Transition Matrix for Markov Chain
import re
from collections import Counter

# Parse text
text = '''It is a truth universally acknowledged...'''.lower()
words = re.findall(r"\w+", text)
unique_words = sorted(set(words))
wordToIndex = {word: idx for idx, word in enumerate(unique_words)}

# Count transitions
transitions = [(words[i], words[i + 1]) for i in range(len(words) - 1)]
transition_counts = Counter(transitions)

# Transition matrix
n_words = len(unique_words)
transition_matrix = np.zeros((n_words, n_words))
for (w1, w2), count in transition_counts.items():
    i, j = wordToIndex[w1], wordToIndex[w2]
    transition_matrix[i, j] = count

# Normalize rows
transition_matrix /= transition_matrix.sum(axis=1, keepdims=True)
print("Transition matrix computed.")

# Problem 5: Sphere Statistics
from sympy import symbols, gamma, pi

# Variance of X1
d = symbols('d')
variance_x1_problem7 = 1 / d

# Epsilon for 99% shell volume
epsilon = symbols('epsilon')
volume_shell = 1 - (1 - epsilon**d)

# Radius for constant volume
r = (gamma(d / 2 + 1) / (pi**(d / 2)))**(1 / d)
print(f"Variance: {variance_x1_problem7}, Radius: {r}")

# Problem 6: Perceptron Algorithm

def perceptron(X, y, max_iter=1000):
    w = np.zeros(X.shape[1] + 1)
    for _ in range(max_iter):
        for i, x in enumerate(X):
            x_ext = np.insert(x, 0, 1)
            if y[i] * np.dot(w, x_ext) <= 0:
                w += y[i] * x_ext
    return w

hat_w = perceptron(X, y)
print("Weight vector:", hat_w)

# Problem 7: Bootstrap Confidence Interval
from random import randint

def bootstrap(data, func, B=1000, alpha=0.05):
    estimates = [func(np.random.choice(data, len(data), replace=True)) for _ in range(B)]
    lower = np.percentile(estimates, 100 * alpha / 2)
    upper = np.percentile(estimates, 100 * (1 - alpha / 2))
    return lower, upper

data = np.random.exponential(scale=10, size=100)
func = lambda x: np.percentile(x, 95)
ci = bootstrap(data, func)
print("95th Percentile CI:", ci)

# Problem 8: Optimization for MLE
from scipy.optimize import minimize_scalar

def neg_log_likelihood(beta):
    return -np.sum(np.log((dataSamples1 / beta**2) * np.exp(-0.5 * (dataSamples1 / beta)**2)))

result = minimize_scalar(neg_log_likelihood, bounds=(0.1, 10), method="bounded")
print("MLE for beta:", result.x)
